In [2]:
%xmode plain
import glob
import cv2
import numpy as np
import time

Exception reporting mode: Plain


In [3]:
suits_dict = {0:'Diamond',1:'Spade', 2:'Heart', 3:'Club'}

In [4]:
def flattener(image, pts, w, h):
    """Flattens an image of a card into a top-down 200x300 perspective.
    Returns the flattened, re-sized, grayed image.
    See www.pyimagesearch.com/2014/08/25/4-point-opencv-getperspective-transform-example/"""
    temp_rect = np.zeros((4,2), dtype = "float32")
    
    s = np.sum(pts, axis = 2)

    tl = pts[np.argmin(s)]
    br = pts[np.argmax(s)]
    
    diff = np.diff(pts, axis = -1)
    tr = pts[np.argmin(diff)]
    bl = pts[np.argmax(diff)]
    
#     pri
    # Need to create an array listing points in order of
    # [top left, top right, bottom right, bottom left]
    # before doing the perspective transform

    if w <= 0.8*h: # If card is vertically oriented
        temp_rect[0] = tl
        temp_rect[1] = tr
        temp_rect[2] = br
        temp_rect[3] = bl

    if w >= 1.2*h: # If card is horizontally oriented
        temp_rect[0] = bl
        temp_rect[1] = tl
        temp_rect[2] = tr
        temp_rect[3] = br

    # If the card is 'diamond' oriented, a different algorithm
    # has to be used to identify which point is top left, top right
    # bottom left, and bottom right.
    
    if w > 0.8*h and w < 1.2*h: #If card is diamond oriented
        # If furthest left point is higher than furthest right point,
        # card is tilted to the left.
        if pts[1][0][1] <= pts[3][0][1]:
            # If card is titled to the left, approxPolyDP returns points
            # in this order: top right, top left, bottom left, bottom right
            temp_rect[0] = pts[1][0] # Top left
            temp_rect[1] = pts[0][0] # Top right
            temp_rect[2] = pts[3][0] # Bottom right
            temp_rect[3] = pts[2][0] # Bottom left

        # If furthest left point is lower than furthest right point,
        # card is tilted to the right
        if pts[1][0][1] > pts[3][0][1]:
            # If card is titled to the right, approxPolyDP returns points
            # in this order: top left, bottom left, bottom right, top right
            temp_rect[0] = pts[0][0] # Top left
            temp_rect[1] = pts[3][0] # Top right
            temp_rect[2] = pts[2][0] # Bottom right
            temp_rect[3] = pts[1][0] # Bottom left
            
        
    maxWidth = 200
    maxHeight = 300

    # Create destination array, calculate perspective transform matrix,
    # and warp card image
    dst = np.array([[0,0],[maxWidth-1,0],[maxWidth-1,maxHeight-1],[0, maxHeight-1]], np.float32)
    M = cv2.getPerspectiveTransform(temp_rect,dst)
    warp = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
#     warp = cv2.cvtColor(warp,cv2.COLOR_BGR2GRAY)
#     cv2.imshow('warp', warp)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
    return warp

In [5]:
def sift_detect(cropped_suit, suits):
    sift = cv2.xfeatures2d.SIFT_create()
    distances = []
    for i in range(len(suits)):
        kp1, des1 = sift.detectAndCompute(cropped_suit, None)
        sift = cv2.xfeatures2d.SIFT_create()
        kp2, des2 = sift.detectAndCompute(suits[i], None)
        bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
        matches = bf.match(des1, des2)
        matches = sorted(matches, key = lambda x:x.distance)
        sums = 0
        for j in range(3):
            sums += matches[j].distance
        distances.append(sums)
    print(" The card suit is:  ", suits_dict[np.argmin(np.array(distances))])
    cropped_suit =  cv2.flip(cropped_suit, 0)
    distances = []
    for i in range(len(suits)):
        kp1, des1 = sift.detectAndCompute(cropped_suit, None)
        sift = cv2.xfeatures2d.SIFT_create()
        kp2, des2 = sift.detectAndCompute(suits[i], None)
        bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
        matches = bf.match(des1, des2)
        matches = sorted(matches, key = lambda x:x.distance)
        sums = 0
        for j in range(3):
            sums += matches[j].distance
        distances.append(sums)
    print(" The card suit is:  ", suits_dict[np.argmin(np.array(distances))])

    

In [6]:
def find_rank(img_suit):
    _, img_suit = cv2.threshold(img_suit,128,255,0)
    temp1 =  cropped_suit[0:int((cropped_suit.shape[0]/2)) , :]
    temp2 = cropped_suit[int(cropped_suit.shape[0]/2 + 1): cropped_suit.shape[0], :]
    
    suits = [cv2.imread(file) for file in glob.glob("suits/*.png")]
    
    nw_pixels1 = temp1[ temp1 >= 128].shape
    nw_pixels2 = temp2[ temp2 >= 128].shape
    facing_up = False
    
    if  nw_pixels1 > nw_pixels2:
        facing_up = True
    
    for i,img in enumerate(suits):
        img = cv2.cvtColor(cv2.resize(img, (img_suit.shape[1], img_suit.shape[0])), cv2.COLOR_BGR2GRAY)
        _,img = cv2.threshold(img,127,255,0)
        suits[i] = img
    
    minimum1 = 1000000
    idx1  = 0   
    ret = 0
    
    minimum2 = 1000000
    idx2 = 0
    
    if facing_up:
        print("Is facing up")
        for j in range(len(suits)):
            ans = np.abs(-img_suit + suits[j])
            ans = ans[ans > 200].shape[0]
#             print(j," - ", ans)
            if ans < minimum2:
                minimum2 = ans
                idx2 = j
        cv2.imshow('query suit', img_suit)
        cv2.imshow('spade', -img_suit + suits[1])
        cv2.imshow('heart2', -img_suit + suits[2])
        cv2.imshow('club', -img_suit + suits[3])
        
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    else:
        print("Is facing down")
        for i in range(len(suits)):
            ans = np.abs((-img_suit + cv2.flip(suits[i], 0)))
            tans = ans[ans > 200].shape[0]
#             if i == 2:
#                 ans = np.abs((-img_suit + suits[i]))
#                 tans = ans[ans > 200].shape[0]         

            if tans < minimum1:
                minimum1 = tans
                idx1 = i
                ret = ans
        cv2.imshow('query suit', img_suit)
        cv2.imshow('spade', -img_suit + cv2.flip(suits[1], 0))
        cv2.imshow('heart', -img_suit + cv2.flip(suits[2], 0))
        cv2.imshow('club', -img_suit + cv2.flip(suits[3], 0))
        
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    if minimum2 < minimum1:   
        return idx2
    else:    
        return idx1

In [11]:
def preprocess_card(contour, img):
    
    peri = cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, 0.01*peri, True)
    pts = np.float32(approx)
    x,y,w,h = cv2.boundingRect(approx)
    
    warp = flattener(img, pts, w, h)
    
#     cv2.imshow('warp', warp)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
    
    ## Card image
    warp = cv2.cvtColor(warp,cv2.COLOR_BGR2GRAY)

    CORNER_WIDTH = 30
    CORNER_HEIGHT = 84
    
    cropped_suit = warp[0:CORNER_HEIGHT, 0:CORNER_WIDTH]
    csuit = cv2.resize(cropped_suit, (0,0), fx=4, fy=4)
    
    white_level = csuit[15,int((CORNER_WIDTH*4)/2)]
    thresh_level = white_level - 30 # CARD_THRESH
    if (thresh_level <= 0):
        thresh_level = 1
        
#     retval, csuit = cv2.threshold(csuit, thresh_level, 255, cv2. THRESH_BINARY_INV)
    
    
    half = int(csuit.shape[0]/2)
        
    return csuit[0:half,:] , csuit[half+1:half*2,:]

In [10]:
mask = cv2.imread('./CV-BlackjackStudent/GOPR0317.MP4-lbl/04873-lbl.png',1)
input_img = cv2.imread('./CV-BlackjackStudent/GOPR0317.MP4-lbl/04873.png',1)
mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(mask,128,255,0)
_, input_img_t = cv2.threshold(input_img,128,255,0)
im2, num_cards, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)


# cimg = cv2.drawContours(input_img, num_cards,-1,(0,255,0),2)
# cv2.imshow('heylo', input_img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

# Read in training card suits

card_value, card_suit = preprocess_card(num_cards[0], input_img)
# cv2.imshow('cropped_suit', cropped_suit)
# cv2.imshow('card value', card_value)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

im2, contours, hierarchy = cv2.findContours(card_value,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
contours = sorted(contours, key=cv2.contourArea,reverse=True)
print("shape: ", card_value.shape )
# print(contours)

x, y, w, h = cv2.boundingRect(contours[0])
print(x,y,w,h)
isolated_rank = card_value[y:y+h, x:x+w]
# cv2.rectangle(card_value, (x,y), (x+w,y+h), (0,255,0),2)
# cv2.drawContours(card_value, contours,-1,(255,255,255),2)
im2, contours, hierarchy = cv2.findContours(card_suit,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
contours = sorted(contours, key=cv2.contourArea,reverse=True)
print("shape: ", card_suit.shape )
# print(contours)

x, y, w, h = cv2.boundingRect(contours[0])
print(x,y,w,h)
isolated_suit = card_suit[y:y+h, x:x+w]

cv2.imshow('card value', isolated_rank)
cv2.imshow('card suit', isolated_suit)
cv2.waitKey(0)
cv2.destroyAllWindows()



# for j in range(len(num_cards)):
#     cropped_suit = 0
#     card_value = 0
#     cropped_suit, card_value = preprocess_card(num_cards[j], input_img)
#     cv2.imshow('cropped_suit', cropped_suit)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
#     #######
#     suits = [cv2.imread(file) for file in glob.glob("suits/*.png")]
#     for i,img in enumerate(suits):
#         img = cv2.cvtColor(cv2.resize(img, (cropped_suit.shape[1], cropped_suit.shape[0])), cv2.COLOR_BGR2GRAY)
#         suits[i] = img
#     #######
    
#     a = find_rank(cropped_suit)
#     print("The suit of the card is: ", suits_dict[a], " value: ", card_value)
#     print("------ SIFT ------")
#     sift_detect(cropped_suit, suits)


shape:  (168, 120)
0 50 78 118
shape:  (167, 120)
0 29 89 98


In [13]:
all_cards = [cv2.imread(file) for file in glob.glob("./CV-BlackjackStudent/GOPR0317.MP4-lbl/test/*.png")]

In [14]:
len(all_cards)

107

In [15]:
x = np.linspace(0, len(all_cards), 54, endpoint=False)
for i in x:
    
    mask = all_cards[int(i)]
    input_img = all_cards[int(i)+1]
    mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(mask,128,255,0)
    _, input_img_t = cv2.threshold(input_img,128,255,0)
    im2, num_cards, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

    card_value, card_suit = preprocess_card(num_cards[0], input_img)
    
#     cv2.imshow('card value', all_cards[0])
#     cv2.imshow('card suit', all_cards[1])
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
    
    ## Isolate the rank
    im2, contours, hierarchy = cv2.findContours(card_value,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=cv2.contourArea,reverse=True)
#     x, y, w, h = cv2.boundingRect(contours[0])
    isolated_rank = card_value[y:y+h, x:x+w]
    
    ## Isolate the suit
    im2, contours, hierarchy = cv2.findContours(card_suit,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=cv2.contourArea,reverse=True)
    
#     x, y, w, h = cv2.boundingRect(contours[0])
    isolated_suit = card_suit[y:y+h, x:x+w]
    
    cv2.imwrite('training_data/' + str(time.time()) + 'A.png', isolated_rank)
    cv2.imwrite('training_data/' + str(time.time()) + 'B.png', isolated_suit)
    


IndexError: list index out of range

In [ ]:
all_cards[0].shape

In [ ]:
pwd